<a href="https://colab.research.google.com/github/ImeshaDilshani/MT---datasset/blob/main/MarinaMT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Install required libraries
!pip install transformers datasets sentencepiece sacrebleu scikit-learn pandas

In [8]:
# Import libraries
import pandas as pd
from datasets import Dataset
from transformers import MarianTokenizer, MarianMTModel, Seq2SeqTrainer, Seq2SeqTrainingArguments
from sklearn.model_selection import train_test_split
import os


In [9]:
# Define file paths (Update with your actual Google Drive file path)
tamil_path = "/content/drive/MyDrive/Research/dataset/MT/2 - 22626/All (Ta-Si) - Tamil.txt"
sinhala_path = "/content/drive/MyDrive/Research/dataset/MT/2 - 22626/All (Ta-Si) - Sinhala.txt"

# Load Tamil and Sinhala text files
with open(tamil_path, encoding="utf-8") as f:
    tamil_lines = f.read().splitlines()

with open(sinhala_path, encoding="utf-8") as f:
    sinhala_lines = f.read().splitlines()

# Trim and validate
min_len = min(len(tamil_lines), len(sinhala_lines))
tamil_lines = tamil_lines[:min_len]
sinhala_lines = sinhala_lines[:min_len]

assert len(tamil_lines) == len(sinhala_lines), "Mismatched line count!"

# Create DataFrame
data = pd.DataFrame({"translation": [{"ta": ta, "si": si} for ta, si in zip(tamil_lines, sinhala_lines)]})

# Shuffle and split
data = data.sample(frac=1, random_state=42).reset_index(drop=True)
train_data, val_data = train_test_split(data, test_size=0.1, random_state=42)

# Convert to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)


In [10]:
# Load pretrained MarianMT model
model_name = "Helsinki-NLP/opus-mt-en-ROMANCE"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Tokenize
max_length = 128

def tokenize_data(example):
    inputs = tokenizer(example["translation"]["ta"], max_length=max_length, padding="max_length", truncation=True)
    targets = tokenizer(example["translation"]["si"], max_length=max_length, padding="max_length", truncation=True)
    inputs["labels"] = targets["input_ids"]
    return inputs

train_dataset = train_dataset.map(tokenize_data, batched=False)
val_dataset = val_dataset.map(tokenize_data, batched=False)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/779k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/799k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.46M [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Map:   0%|          | 0/20364 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]

Map:   0%|          | 0/2263 [00:00<?, ? examples/s]

In [11]:
# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/Research/dataset/MT/2 - 22626/marianmt-ta-si",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=5,
    predict_with_generate=True,
    logging_dir="/content/drive/MyDrive/Research/dataset/MT/2 - 22626/logs",
    logging_steps=50,
)

# Trainer setup
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)


<ipython-input-11-d7cb0b770e4e>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [12]:
# Train the model
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: imeshadilshani212 (imeshadilshani212-university-of-kelaniya) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,1.581400,1.532655
2,1.389500,1.336736
3,1.287100,1.232149
4,1.216300,1.183302
5,1.218500,1.167109


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=6365, training_loss=1.4324592695213731, metrics={'train_runtime': 2475.1512, 'train_samples_per_second': 41.137, 'train_steps_per_second': 2.572, 'total_flos': 3451531516968960.0, 'train_loss': 1.4324592695213731, 'epoch': 5.0})

In [13]:
# Save final model and tokenizer
model.save_pretrained("/content/drive/MyDrive/Research/dataset/MT/2 - 22626/marianmt-ta-si")
tokenizer.save_pretrained("/content/drive/MyDrive/Research/dataset/MT/2 - 22626/marianmt-ta-si")

print("✅ Training complete. Model saved in Google Drive at 'marianmt-ta-si'")


✅ Training complete. Model saved in Google Drive at 'marianmt-ta-si'


In [20]:
! pip install sacremoses

In [21]:
from transformers import MarianMTModel, MarianTokenizer

# Load trained model and tokenizer from Google Drive
model_path = "/content/drive/MyDrive/Research/dataset/MT/2 - 22626/marianmt-ta-si"
tokenizer = MarianTokenizer.from_pretrained(model_path)
model = MarianMTModel.from_pretrained(model_path)

# Function to translate Tamil to Sinhala
def translate_ta_to_si(tamil_text):
    # Tokenize input
    inputs = tokenizer(tamil_text, return_tensors="pt", padding=True, truncation=True)
    # Generate translation
    translated = model.generate(**inputs)
    # Decode output
    sinhala_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return sinhala_text

# 🔎 Example: Test with your own Tamil sentence
test_sentence = "இந்த பிரமாணம் செய்யும் நிகழ்வானது உண்மையிலேயே மிக எளிமையான வைபவமாகும் ."  # "How are you?"
translated_output = translate_ta_to_si(test_sentence)

print("Tamil:", test_sentence)
print("Sinhala:", translated_output)


Tamil: இந்த பிரமாணம் செய்யும் நிகழ்வானது உண்மையிலேயே மிக எளிமையான வைபவமாகும் .
Sinhala: මෙම ප්රශ්නයේ ප්රතිපත්තියක් ප්රතිකාශ කරන විසින් ප්රශාශ කරන ඇත
